In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

In [31]:
load_dotenv("../.env")

LEAF_TABLE_NAME = "form_127"
LEAF_DB_NAME = "eform_data"
LEAF_DB_USER = "kamalleaf"
LEAF_DB_PASS = os.getenv("LEAF_DB_PASS")
LEAF_DB_HOST = "13.214.63.7"

LLM = ChatOpenAI(model="gpt-4-turbo")

In [33]:
db = SQLDatabase.from_uri(
    f"mysql+mysqlconnector://{LEAF_DB_USER}:{LEAF_DB_PASS}@{LEAF_DB_HOST}/{LEAF_DB_NAME}"
)

In [34]:
print(db.dialect)
print(db.get_usable_table_names())

mysql
['eform_titles', 'form_127', 'form_131', 'form_145', 'form_155', 'form_156', 'form_158', 'form_170', 'form_171', 'form_data_testing']


In [14]:
SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
DO NOT create any new table to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.
"""

In [15]:
SQL_FUNCTIONS_SUFFIX = f"""I should only look in {LEAF_TABLE_NAME} table.  Then I should query the schema"""

In [35]:
agent_executor = create_sql_agent(
    LLM, db=db, agent_type="openai-tools", verbose=True, prefix=SQL_PREFIX
)

In [36]:
agent_executor

AgentExecutor(name='SQL Agent Executor', verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessage(content='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct mysql query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.\nYou can order the results by 

In [37]:
agent_executor.invoke({"input": "how many meetings are there?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


eform_titles, form_127, form_131, form_145, form_155, form_156, form_158, form_170, form_171, form_data_testing
Invoking: `sql_db_schema` with `{'table_names': 'form_127, form_131, form_145, form_155, form_156, form_158, form_170, form_171'}`



CREATE TABLE form_127 (
	id INTEGER UNSIGNED NOT NULL AUTO_INCREMENT, 
	id_digital_form_io INTEGER NOT NULL, 
	id_digital_form_io_submission INTEGER NOT NULL, 
	id_user INTEGER NOT NULL, 
	id_group INTEGER NOT NULL, 
	entry_date DATETIME NOT NULL, 
	last_submit_date DATETIME NOT NULL, 
	PRIMARY KEY (id)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_unicode_ci

/*
3 rows from form_127 table:
id	id_digital_form_io	id_digital_form_io_submission	id_user	id_group	entry_date	last_submit_date
1	127	149	29741	114	2024-06-25 10:19:04	2024-06-25 10:19:04
2	127	150	29741	114	2024-06-26 01:57:06	2024-06-26 01:57:06
3	127	151	29741	114	2024-06-26 02:02:01	2024-

{'input': 'how many meetings are there?',
 'output': 'There are a total of 3 meetings recorded in the database.'}

In [40]:
agent_executor.invoke(
    {"input": "how many LEAF meetings are there? Show me the content of one of them"}
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


eform_titles, form_127, form_131, form_145, form_155, form_156, form_158, form_170, form_171, form_data_testing
Invoking: `sql_db_schema` with `{'table_names': 'form_127, form_131, form_145, form_155, form_156, form_158, form_170, form_171'}`



CREATE TABLE form_127 (
	id INTEGER UNSIGNED NOT NULL AUTO_INCREMENT, 
	id_digital_form_io INTEGER NOT NULL, 
	id_digital_form_io_submission INTEGER NOT NULL, 
	id_user INTEGER NOT NULL, 
	id_group INTEGER NOT NULL, 
	entry_date DATETIME NOT NULL, 
	last_submit_date DATETIME NOT NULL, 
	PRIMARY KEY (id)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_unicode_ci

/*
3 rows from form_127 table:
id	id_digital_form_io	id_digital_form_io_submission	id_user	id_group	entry_date	last_submit_date
1	127	149	29741	114	2024-06-25 10:19:04	2024-06-25 10:19:04
2	127	150	29741	114	2024-06-26 01:57:06	2024-06-26 01:57:06
3	127	151	29741	114	2024-06-26 02:02:01	2024-

{'input': 'how many LEAF meetings are there? Show me the content of one of them',
 'output': 'There are a total of 3 LEAF meetings recorded in the database.\n\nHere is the content of one of the LEAF meetings:\n\n- **Property Details**: THE TWO CONDO, Tingkat 18, Menara TH Perdana, 1001, Jln Sultan Ismail, 50250 Kuala Lumpur, Federal Territory of Kuala Lumpur\n- **Meeting Subject**: LEAF Meeting\n- **Date and Time of Meeting**: May 24, 2024, at 12:00 PM\n- **Attendees and Roles**:\n  - John (Mr.), T-04-04, Chairman, Attendance: 1/1\n  - Moi (Mr.), T-05-05, Treasurer, Attendance: 1/1\n  - Lisa (Ms.), T-06-06, Secretary, Attendance: 1/1\n  - Lin (Mr.), T-08-08, Treasurer, Attendance: 1/1\n  - Erno (Ms.), T-11-11, Treasurer, Attendance: 1/1\n  - Moon (Mr.), T-12-12, Committee, Attendance: 1/1\n\n- **Meeting Proceedings**:\n  - **Approval of Previous Minutes**: The minutes from the May 24, 2024 meeting were approved without amendments.\n  - **Project Updates**: Project Alpha: Lin reported t